In [45]:
import requests
import json
import os
import time
from datetime import datetime, timedelta

In [51]:
def requisicao(query,one_hour_ago, current_time):   
    # URL da API webhook
    url = "http://192.168.15.18:8000/webhook"
    
    # Dados a serem enviados na requisição
    data = {
        "query": query,
        "from":one_hour_ago,
        "to":current_time
    }

    # Enviando a requisição POST para a API
    response = requests.post(url, json=data)


    # Verificando o status da resposta
    if response.status_code == 200:

        # Geração do arquivo JSON
        webhook_data = response.json()

        if webhook_data["totalResults"] > 0:
            
            webhook_data = webhook_data["articles"]

            today = datetime.now().strftime('%Y-%m-%d')
            hm = datetime.now().strftime('%H-%M')

            folder_path = f'noticias/{today}'
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            with open(f'{folder_path}/{hm}-news.json', 'w') as f:
                json.dump(webhook_data, f)
            print("Novas notícias foram encontradas e salvas:", webhook_data)

        else:
            print("Não há noticias novas com essa Query")
        
    else:
        # Caso contrário, houve um erro na requisição
        print("Erro ao enviar webhook.")


while True:
    query = "dengue"

    dia_anterior = (datetime.now() - timedelta(days=1))
    current_time = dia_anterior.strftime('%Y-%m-%dT%H:%M:%S')
    one_hour_ago = (dia_anterior - timedelta(hours=1)).strftime('%Y-%m-%d')
    
    if current_time[-5:] == "00:00":
        requisicao(query,one_hour_ago, current_time)
        time.sleep(10) 


